In [132]:
import re
import string
import contractions
from dataset import Dataset
from collections import defaultdict

In [127]:
import importlib
importlib.reload(dataset_constants) # reload to update changes to the file
from dataset_constants import *

# Load Dataset

In [3]:
dataset = Dataset(train_path=TRAIN_DATASET_PATH,
                  val_path=VALIDATION_DATASET_PATH, 
                  test_path=TEST_DATASET_PATH)
dataset.build()

In [4]:
X_train = dataset.get_features(split_type="train")
Y_train = dataset.get_labels(split_type="train")

print(X_train[:5])
print(Y_train[:5])
print(len(X_train))
print(len(Y_train))

['Inaasahan na ni Vice President Jejomar Binay na may mga taong... https://t.co/SDytgbWiLh', 'Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA NGA DI STRAIGHT', 'Salamat sa walang sawang suporta ng mga taga makati! Ang Pagbabalik Binay In Makati #OnlyBinayInMakatiSanKaPa https://t.co/iwAOdtZPRE', '@rapplerdotcom putangina mo binay TAKBO PA', 'Binay with selective amnesia, forgetting about the past six years he spent preparing to be president.  #PiliPinasDebates2016']
[0, 1, 0, 1, 0]
10000
10000


In [5]:
X_val = dataset.get_features(split_type="val")
Y_val = dataset.get_labels(split_type="val")

print(X_val[:5])
print(Y_val[:5])
print(len(X_val))
print(len(Y_val))

['Escudero denies betraying Poe after meeting with Binay |https://t.co/sKlXTIhHJa - Kare-kare at sinampalukan ang topic. Walang balimbing?', 'Hndi ko makita yung sa one more chance saka kay binay sa fb. Haist.', "Mar Roxas is now addressing the crowd gathered at Pasay City's Ulat sa Barangay 2016  https://t.co/VruZyJ2e2H", '@ImYourBaeMax perfect! Para makaharap ni Duterte ang mga Binay at makatikim ng mura #^%* i#*', '#OnlyBinayPriority4Ps Wag nating hayaan na maloko tayo ng mga pulitikong yan. Kay Binay na tayo']
[0, 1, 0, 0, 0]
4232
4232


In [6]:
X_test = dataset.get_features(split_type="test")
Y_test = dataset.get_labels(split_type="test")

print(X_test[:5])
print(Y_test[:5])
print(len(X_test))
print(len(Y_test))

['Unshaded votes and votes for Mayor Duterte goes to Mar Roxas according to some reports of ballot tests.  #AyawSaDILAW', '#NoMoreChance https://t.co/msaaUGv0bS', "@itsmanj well there's other good choices like Duterte or Poe. But both of them are still undecided, I think? :( :(", 'Nognog. Pandak. Laki sa hirap. Pero corrupt. Yan si Binay!!!', 'Ex-Binay aide turns tables on Mercado | https://t.co/nyySAo54rL']
[1, 1, 0, 1, 0]
4232
4232


In [7]:
X = dataset.get_features()
Y = dataset.get_labels()
print(len(X))
print(len(Y))
print(X[:5])
print(Y[:5])

18464
18464
['Inaasahan na ni Vice President Jejomar Binay na may mga taong... https://t.co/SDytgbWiLh', 'Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA NGA DI STRAIGHT', 'Salamat sa walang sawang suporta ng mga taga makati! Ang Pagbabalik Binay In Makati #OnlyBinayInMakatiSanKaPa https://t.co/iwAOdtZPRE', '@rapplerdotcom putangina mo binay TAKBO PA', 'Binay with selective amnesia, forgetting about the past six years he spent preparing to be president.  #PiliPinasDebates2016']
[0, 1, 0, 1, 0]


# Helper Functions

In [115]:
def count_texts(texts, pattern_type):
    # Define the regex pattern
    if pattern_type == 'url':
        pattern = r'https?://\S+|www\.\S+'
    elif pattern_type == 'username':
        pattern = r'@\S+'
    elif pattern_type == 'numeric':
        pattern = r'\b\d+\b'
    elif pattern_type == 'html_tags':
        pattern = r'<.*?>+'
    elif pattern_type == 'newlines':
        pattern = r'\n'
    elif pattern_type == 'punctuation':
        pattern = '[%s]' % re.escape(string.punctuation)
    elif pattern_type == 'emoji':
        pattern = re.compile("["
                               "\U0001F600-\U0001F64F"  # emoticons
                               "\U0001F300-\U0001F5FF"  # symbols & pictographs
                               "\U0001F680-\U0001F6FF"  # transport & map symbols
                               "\U0001F700-\U0001F77F"  # alchemical symbols
                               "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               "\U0001FA00-\U0001FA6F"  # Chess Symbols
                               "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               "\U00002702-\U000027B0"  # Dingbats
                               "\U000024C2-\U0001F251" 
                               "]+", flags=re.UNICODE)


    count = 0

    # Iterate through each text and check for the presence of the pattern
    for text in texts:
        if re.search(pattern, text):
            count += 1
    
    return count

In [14]:
def clean_texts(texts, pattern_type):
    # Define the regex pattern
    if pattern_type == 'url':
        pattern = r'https?://\S+|www\.\S+'
    elif pattern_type == 'username':
        pattern = r'@\S+'
    elif pattern_type == 'numeric':
        pattern = r'\b\d+\b'
    elif pattern_type == 'html_tags':
        pattern = r'<.*?>+'
    elif pattern_type == 'newlines':
        pattern = r'\n'
    elif pattern_type == 'punctuation':
        pattern = '[%s]' % re.escape(string.punctuation)


    # Remove URLs from each text in the list
    return [re.sub(pattern, '', text) for text in texts]

In [51]:
custom_contractions = {
    "gov't": "government",
    "s'ya": "siya",
    "sa'yo": "sa iyo",
    "ika'y": "ikaw ay",
    "everybody's": "everybody is",
    "mo'ko": "mo ako",
    "ba't": "bakit",
    "sila'y": "sila ay",
    "aba'y": "aba ay",
    "ito'y": "ito ay",
    "mgm't": "management",
    "shut'up": "shut up",
    "you're": "you are",
    "umano'y": "umano ay",
    "kaya't": "kaya at",
    "n'ya": "niya",
    "le'me": "let me",
    "c'mon": "common",
    "isa't": "isa at",
    "ako'y": "ako ay",
    "toyo't": "toyo at",
    "na'to": "na ito",
    "n'yo": "niyo"
}

def expand_all_contractions(text, custom_dict):
    # First, expand using the default package
    expanded_text = contractions.fix(text)
    # Now apply custom contractions
    for key, value in custom_dict.items():
        expanded_text = expanded_text.replace(key, value)
    return expanded_text

In [61]:
def count_contractions(texts):
    # Regular expression to match contractions
    contraction_pattern = r"\b\w+['’]\w+\b"
    count = 0

    # Iterate through each text and count contractions
    for text in texts:
        # Find all instances of the pattern
        contracted_words = re.findall(contraction_pattern, text)
        count += len(contracted_words)
    
    return count

In [109]:
# Function to read stopwords from file and save into a list
def read_stopwords(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        stopwords_list = [word.strip() for word in file.readlines()]
    return stopwords_list

# Read English stopwords
english_stopwords = read_stopwords(STOPWORDS_ENGLISH_PATH)

# Read Tagalog stopwords
tagalog_stopwords = read_stopwords(STOPWORDS_TAGALOG_PATH)

# Combine stopwords
combined_stopwords = set(english_stopwords + tagalog_stopwords)

In [110]:
def remove_stopwords(text, stopwords):
    # Split the text into words
    words = text.split()
    # Filter out the stopwords
    filtered_words = [word for word in words if word not in stopwords]
    # Join the filtered words back into a single string
    return ' '.join(filtered_words)

In [136]:
def remove_duplicates(X, y):
    # Create a dictionary to store indices of non-unique tweets
    non_unique_indices = defaultdict(list)

    # Iterate over the feature data X and store the indices of non-unique tweets
    for i, tweet in enumerate(X):
        non_unique_indices[tweet].append(i)

    # Identify the indices of duplicates
    duplicate_indices = [indices for indices in non_unique_indices.values() if len(indices) > 1]

    # Flatten the list of duplicate indices
    duplicate_indices = [idx for sublist in duplicate_indices for idx in sublist]

    # Remove duplicates from X and y
    X_unique = [X[i] for i in range(len(X)) if i not in duplicate_indices]
    y_unique = [y[i] for i in range(len(y)) if i not in duplicate_indices]

    # Now, count how many removed items belong to each set
    removed_train_count = sum(1 for idx in duplicate_indices if idx < split_sizes[0])
    removed_validation_count = sum(1 for idx in duplicate_indices if split_sizes[0] <= idx < split_sizes[0] + split_sizes[1])
    removed_test_count = sum(1 for idx in duplicate_indices if split_sizes[0] + split_sizes[1] <= idx)
    total_removed = removed_train_count + removed_validation_count + removed_test_count

    print(f"Removed {total_removed} non-unique tweets.")
    print("Removed from train set:", removed_train_count)
    print("Removed from validation set:", removed_validation_count)
    print("Removed from test set:", removed_test_count)

    return X_unique, y_unique, non_unique_indices


In [150]:
def cleaning_pipeline(X, Y):
    # Remove URLs
    print(f"Text with URLs: {count_texts(X, 'url')}")
    X_url_removed = clean_texts(X, 'url')
    print(f"Removed URLs. New count of text with URLs: {count_texts(X_url_removed, 'url')}\n")

    # Remove usernames
    print(f"Text with usernames: {count_texts(X_url_removed, 'username')}")
    X_username_removed = clean_texts(X_url_removed, 'username')
    print(f"Removed usernames. New count of text with usernames: {count_texts(X_username_removed, 'username')}\n")

    # Remove words that are completely numbers
    print(f"Text with numeric words: {count_texts(X_username_removed, 'numeric')}")
    X_numeric_removed = clean_texts(X_username_removed, 'numeric')
    print(f"Removed numeric words. New count of text with numeric words: {count_texts(X_numeric_removed, 'numeric')}\n")

    # Remove HTML tags
    print(f"Text with HTML tags: {count_texts(X_numeric_removed, 'html_tags')}")
    X_html_removed = clean_texts(X_numeric_removed, 'html_tags')
    print(f"Removed HTML tags. New count of text with HTML tags: {count_texts(X_html_removed, 'html_tags')}\n")

    # Remove newlines
    print(f"Text with newlines: {count_texts(X_html_removed, 'newlines')}")
    X_newlines_removed = clean_texts(X_html_removed, 'newlines')
    print(f"Removed newlines. New count of text with newlines: {count_texts(X_newlines_removed, 'newlines')}\n")

    # Lowercase all texts
    X_lowercased = [text.lower() for text in X_newlines_removed]
    print(f"Converted all texts to lowercase.\n")

    # Expand contractions
    print(f"Identified {count_contractions(X_lowercased)} contractions. Attempting to expand some of them.")
    X_expanded = [expand_all_contractions(text, custom_contractions) for text in X_lowercased]
    print(f"Expanded contractions. Remaining contractions: {count_contractions(X_expanded)} (Some are possessive.)\n")

    # Remove punctuations - CAN CONSIDER NOT REMOVING
    print(f"Text with punctuations: {count_texts(X_expanded, 'punctuation')}")
    X_punctuation_removed = clean_texts(X_expanded, 'punctuation')
    print(f"Removed punctuations. New count of text with punctuations: {count_texts(X_punctuation_removed, 'punctuation')}\n")

    # Remove stop words - both English and Filipino
    X_no_stopwords = [remove_stopwords(text, combined_stopwords) for text in X_punctuation_removed]
    print("Removed stopwords from all texts.\n")

    # # Remove non-unique tweets
    # X_unique, Y_unique, duplicated_tweets = remove_duplicates(X_no_stopwords, Y)
    
    return X_no_stopwords
    # return X_unique, Y_unique, duplicated_tweets


Other cleaning steps that can be explored:
- Handle non-unique tweets (PENDING! CRUCIAL.)
- Consider preserving certain punctuations?
- Spelling Corrections
- Handling Slangs and Abbreviations
- Stemming? (reduce words to their root word)
- Handling emojis and special characters (found no emojis)

# Data Cleaning

In [151]:
cleaned_X = cleaning_pipeline(X,Y)

Text with URLs: 5016
Removed URLs. New count of text with URLs: 0

Text with usernames: 3950
Removed usernames. New count of text with usernames: 0

Text with numeric words: 1260
Removed numeric words. New count of text with numeric words: 0

Text with HTML tags: 0
Removed HTML tags. New count of text with HTML tags: 0

Text with newlines: 0
Removed newlines. New count of text with newlines: 0

Converted all texts to lowercase.

Identified 3016 contractions. Attempting to expand some of them.
Expanded contractions. Remaining contractions: 1026 (Some are possessive.)

Text with punctuations: 16779
Removed punctuations. New count of text with punctuations: 0

Removed stopwords from all texts.



In [152]:
print(len(cleaned_X))
print(len(Y))

18464
18464


In [153]:
dataset.save_to_file(cleaned_X, Y, CLEANED_DATASET_PATH)

Data saved to cleaned_dataset.pkl


# Load Cleaned Dataset from Path

In [154]:
dataset = Dataset(full_data_path=CLEANED_DATASET_PATH,
                  from_scratch=False,
                  split_sizes=[10000, 4232, 4232])
dataset.build()

Data loaded from cleaned_dataset.pkl


In [155]:
X_train = dataset.get_features(split_type="train")
Y_train = dataset.get_labels(split_type="train")

print(X_train[:5])
print(Y_train[:5])
print(len(X_train))
print(len(Y_train))

['inaasahan vice president jejomar binay taong', 'mar roxas tang ina tuwid daan daw eh sya nga di straight', 'salamat sawang suporta taga makati pagbabalik binay makati onlybinayinmakatisankapa', 'putangina mo binay takbo', 'binay selective amnesia forgetting past six years spent preparing president pilipinasdebates2016']
[0, 1, 0, 1, 0]
10000
10000


In [156]:
X_val = dataset.get_features(split_type="val")
Y_val = dataset.get_labels(split_type="val")

print(X_val[:5])
print(Y_val[:5])
print(len(X_val))
print(len(Y_val))

['escudero denies betraying poe meeting binay karekare sinampalukan topic balimbing', 'hndi yung one chance saka kay binay fb haist', 'mar roxas addressing crowd gathered pasay citys ulat barangay', 'perfect makaharap duterte binay makatikim mura', 'onlybinaypriority4ps wag nating hayaan maloko pulitikong yan kay binay']
[0, 1, 0, 0, 0]
4232
4232


In [157]:
X_test = dataset.get_features(split_type="test")
Y_test = dataset.get_labels(split_type="test")

print(X_test[:5])
print(Y_test[:5])
print(len(X_test))
print(len(Y_test))

['unshaded votes votes mayor duterte goes mar roxas according reports ballot tests ayawsadilaw', 'nomorechance', 'well good choices like duterte poe still undecided think', 'nognog pandak laki hirap corrupt yan si binay', 'exbinay aide turns tables mercado']
[1, 1, 0, 1, 0]
4232
4232
